# Emojify! 

We will use word vector representations to build an Emojifier. 

Have you ever wanted to make your text messages more expressive? Your emojifier app will help you do that. 
So rather than writing:
>"Congratulations on the promotion! Let's get coffee and talk. Love you!"   

The emojifier can automatically turn this into:
>"Congratulations on the promotion! 👍 Let's get coffee and talk. ☕️ Love you! ❤️"

* The model will take as input a sentence (such as "Let's go see the baseball game tonight!") and finds the most appropriate emoji to be used with this sentence (⚾️).

1.  First model is a baseline model (Emojifier-V1) using word embeddings.
2. Then a more sophisticated model (Emojifier-V2) that further incorporates an LSTM. 

In [6]:
!pip install emoji --upgrade
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
%matplotlib inline

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)


## 1 - Baseline model: Emojifier-V1

### 1.1 - Dataset EMOJISET


Dataset is (X, Y) where:
- X contains 127 sentences (strings).
- Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence.

<img src="https://github.com/varangrai/Coursera-Coursework/blob/master/Emojify/images/data_set.png?raw=true" style="width:700px;height:300px;">
<caption><center> **Figure 1**: EMOJISET - a classification problem with 5 classes. A few examples of sentences are given here. </center></caption>

* X_train: It is an array containing strings
* Y_train: an array containing labels

In [7]:
X_train, Y_train = read_csv('/content/train_emoji.csv') 
X_test, Y_test = read_csv('/content/tesss.csv')

In [8]:
maxLen = len(max(X_train, key=len).split()) # maximum length of a sentence

In [6]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


### 1.2 - Overview of the Emojifier-V1

<center>
<img src="https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/images/image_1.png?raw=true" style="width:900px;height:300px;">
<caption><center> **Figure 2**: Baseline model (Emojifier-V1).</center></caption>
</center>


#### Inputs and outputs
* The input of the model is a string corresponding to a sentence (e.g. "I love you). 
* The output will be a probability vector of shape (1,5), (there are 5 emojis to choose from).
* The (1,5) probability vector is passed to an argmax layer, which extracts the index of the emoji with the highest probability.

#### One-hot encoding
* Converting labels to one-hot encoding
* Each row is a one-hot vector giving the label of one example.


In [9]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

In [10]:
idx = 50
print(f"Sentence '{X_train[50]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [1. 0. 0. 0. 0.]


### 1.3 - Implementing Emojifier-V1

As shown in Figure 2 (above), the first step is to:
* Convert each word in the input sentence into their word vector representations.
* Then take an average of the word vectors. 

Run the following cell to load the `word_to_vec_map`, which contains all the vector representations.

In [11]:
!pip install kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d watts2/glove6b50dtxt
!unzip \*.zip

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
glove6b50dtxt.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  glove6b50dtxt.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [12]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/glove.6B.50d.txt')

- `word_to_index`: dictionary mapping from words to their indices in the vocabulary 
    - (400,001 words, with the valid indices ranging from 0 to 400,000)
- `index_to_word`: dictionary mapping from indices to their corresponding words in the vocabulary
- `word_to_vec_map`: dictionary mapping words to their GloVe vector representation.


In [13]:
word = "cucumber"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


`sentence_to_avg()`
1. Convert every sentence to lower-case, then split the sentence into a list of words. 

2. For each word in the sentence, access its GloVe representation.
    * Then take the average of all of these word vectors.
    
    

In [14]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    words = sentence.lower().split()
    avg = [word_to_vec_map[i] for i in words]
    avg = np.sum(np.array(avg), axis=0)/len(avg)
    return avg

In [15]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = \n", avg)

avg = 
 [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


In [16]:
x_train = np.array([sentence_to_avg(sent,word_to_vec_map) for sent in X_train])
x_test = np.array([sentence_to_avg(sent,word_to_vec_map) for sent in X_test])

In [17]:
import tensorflow as tf

In [18]:
def make_model(embed_dims,):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(64,input_shape=(embed_dims,)),)
  model.add(tf.keras.layers.Dense(5, activation = 'softmax'))
  return model
model = make_model(50)
model.summary()  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                3264      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 3,589
Trainable params: 3,589
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(0.1), 
              metrics=['acc'])
model.fit(x_train,Y_oh_train, batch_size=x_train.shape[0] ,validation_data=(x_test, Y_oh_test), epochs=500)

Epoch 1/500
1/1 [==============================] - 0s 218ms/step - loss: 2.0934 - acc: 0.1515 - val_loss: 4.0485 - val_acc: 0.2321
Epoch 2/500
1/1 [==============================] - 0s 38ms/step - loss: 4.5063 - acc: 0.2955 - val_loss: 5.2416 - val_acc: 0.3929
Epoch 3/500
1/1 [==============================] - 0s 38ms/step - loss: 5.0184 - acc: 0.4091 - val_loss: 12.3764 - val_acc: 0.2143
Epoch 4/500
1/1 [==============================] - 0s 48ms/step - loss: 11.1612 - acc: 0.2803 - val_loss: 15.1903 - val_acc: 0.3571
Epoch 5/500
1/1 [==============================] - 0s 40ms/step - loss: 13.8871 - acc: 0.3712 - val_loss: 15.3204 - val_acc: 0.1429
Epoch 6/500
1/1 [==============================] - 0s 53ms/step - loss: 13.3559 - acc: 0.2121 - val_loss: 4.7409 - val_acc: 0.3750
Epoch 7/500
1/1 [==============================] - 0s 45ms/step - loss: 3.9665 - acc: 0.4318 - val_loss: 2.9647 - val_acc: 0.5357
Epoch 8/500
1/1 [==============================] - 0s 39ms/step - loss: 2.9279 - ac

### 1.4 - Examining test set performance 

* Note that the `predict` function used here is defined in emo_util.spy.

* Random guessing would have had 20% accuracy given that there are 5 classes. (1/5 = 20%).
* This is pretty good performance after training on only 127 examples. 


#### The model matches emojis to relevant words
In the training set, the algorithm saw the sentence 
>"*I love you*" 

with the label ❤️. 




In [20]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

In [21]:
X_my_sentences = ["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"]
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = np.argmax(model.predict(np.array([sentence_to_avg(i, word_to_vec_map) for i in X_my_sentences])), axis=-1)
#print_predictions(X_my_sentences, pred)

In [22]:
temp = [label_to_emoji(i) for i in pred]
for i,j in enumerate(temp):
  print(X_my_sentences[i],j)

i adore you ❤️
i love you ❤️
funny lol 😄
lets play with a ball ⚾
food is ready 🍴
not feeling happy 😄



#### Confusion matrix
* Printing the confusion matrix can also help understand which classes are more difficult for your model. 
* A confusion matrix shows how often an example whose label is one class ("actual" class) is mislabeled by the algorithm with a different class ("predicted" class).

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(Y_oh_test,axis=-1), np.argmax(model.predict(x_test), axis=-1))

array([[ 5,  1,  0,  1,  0],
       [ 0,  8,  0,  0,  0],
       [ 4,  0, 14,  0,  0],
       [ 0,  0,  1, 15,  0],
       [ 0,  0,  0,  0,  7]])

## 2 - Emojifier-V2: Using LSTMs in Keras: 

* This model will be able to account for the word ordering. 
* Emojifier-V2 will continue to use pre-trained word embeddings to represent words.
* We will feed word embeddings into an LSTM.
* The LSTM will learn to predict the most appropriate emoji. 


In [24]:
import numpy as np
from keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


### 2.1 - Overview of the model

Here is the Emojifier-v2 you will implement:

<img src="https://github.com/varangrai/Coursera-Coursework/blob/master/Emojify/images/emojifier-v2.png?raw=true" style="width:700px;height:400px;"> <br>
<caption><center> **Figure 3**: Emojifier-V2. A 2-layer LSTM sequence classifier. </center></caption>



In [25]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    X_indices=[]
    for sent in X:
      temp=[]
      for word in sent.split():
        temp.append(word_to_index[word.lower()])
      X_indices.append(temp)
    X_indices = np.array(X_indices)
    pad_seq = pad_sequences(X_indices, max_len, padding='post', truncating='post')
    return pad_seq

In [26]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])

X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[155345 225122      0      0      0]
 [220930 286375  69714      0      0]
 [151204 192973 302254 151349 394475]]


### Build embedding layer

* The embedding layer takes as input a list of word indices.
    * `sentences_to_indices()` creates these word indices.
* The embedding layer will return the word embeddings for a sentence. 


In [27]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)
    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)

    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = tf.keras.layers.Embedding(vocab_len, emb_dim, weights=[emb_matrix], trainable=False)
    
    return embedding_layer

## 2.3 Building the Emojifier-V2


In [28]:

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = tf.keras.layers.Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices) 
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = tf.keras.layers.LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = tf.keras.layers.Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = tf.keras.layers.LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = tf.keras.layers.Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = tf.keras.layers.Dense(5)(X)
    # Add a softmax activation
    X = tf.keras.layers.Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = tf.keras.models.Model(inputs=sentence_indices, outputs=X)
    
    return model

In [29]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645   

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [33]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32,shuffle=True)

Epoch 1/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4863 - accuracy: 0.3030
Epoch 2/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4275 - accuracy: 0.3636
Epoch 3/50
5/5 [==============================] - 0s 8ms/step - loss: 1.3161 - accuracy: 0.5076
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 1.2125 - accuracy: 0.5303
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - loss: 1.0795 - accuracy: 0.6288
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 0.9205 - accuracy: 0.7197
Epoch 7/50
5/5 [==============================] - 0s 8ms/step - loss: 0.8590 - accuracy: 0.6894
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 0.7271 - accuracy: 0.7727
Epoch 9/50
5/5 [==============================] - 0s 9ms/step - loss: 0.7133 - accuracy: 0.7348
Epoch 10/50
5/5 [==============================] - 0s 9ms/step - loss: 0.5755 - accuracy: 0.8182
Epoch 11/50
5/5 [======================

In [34]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 0s 4ms/step - loss: 0.5781 - accuracy: 0.8036

Test accuracy =  0.8035714030265808


In [35]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😞 prediction: work is hard	😄
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😞 prediction: work is horrible	😄
Expected emoji:🍴 prediction: any suggestions for dinner	😄
Expected emoji:❤️ prediction: I love taking breaks	😞
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	😄
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:❤️ prediction: I love you to the stars and back	😄
Expected emoji:😞 prediction: go away	⚾
Expected emoji:🍴 prediction: I did not have breakfast ❤️


custom example

In [ ]:
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

## LSTM version accounts for word order
* Previously, Emojify-V1 model did not correctly label "not feeling happy," but our implementation of Emojiy-V2 got it right. 

* The current model still isn't very robust at understanding negation (such as "not happy")
    * This is because the training set is small and doesn't have a lot of examples of negation. 
    * But if the training set were larger, the LSTM model would be much better than the Emojify-V1 model at understanding such complex sentences. 
